In [1]:
!pip install pyarabic
!git clone https://github.com/moaaztaha/Arabic-English-Translation-Transformers

Cloning into 'Arabic-English-Translation-Transformers'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 39 (delta 13), reused 34 (delta 8), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [41]:
# modules
import random
import string
import re
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import pandas as pd
from tqdm import tqdm


import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, strip_tatweel

### Data Preprocessing 

In [42]:
ar = pd.read_table('../input/ar-en-translation-small/ArabicNewData.txt', delimiter='\\n', names=['ar'])
en = pd.read_table('../input/ar-en-translation-small/EnglishNewData.txt', delimiter='\\n', names=['en'])

en['ar'] = ar['ar']
df = en.copy()
df = df.iloc[:35118]

/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [43]:
morphs = [strip_tashkeel, strip_tatweel]

def fix_ar(sent):
  sent = split_al_sent(sent)
  tokens = araby.tokenize(sent, morphs=morphs)
  sent = araby.normalize_hamza(' '.join(tokens), method='tasheel')
  return sent

In [44]:
def split_al(word):
    if word.startswith('ال'):
        return word[:2], word[2:]
    else: 
        return word

def split_al_sent(sent):
    ww = []
    for word in sent.split():
        out = split_al(word)
        if type(out) is tuple:
            for w in out:
                ww.append(w)
        else:
            ww.append(word)
    return ' '.join(w for w in ww)

In [45]:
df['ar'] = df.apply(lambda row: fix_ar(row.ar), axis=1)

In [46]:
text_pairs = []
for idx, row in df.iterrows():
  en, ar = row['en'], row['ar']
  ar = "[start] " + ar + " [end]"
  text_pairs.append((en, ar))

In [47]:
for idx, row in df.iterrows():
  if len(row.ar.split()) < 1:
    print(row.ar, '\n*')
    print(row.en)

In [48]:
for _ in range(2):
    print(random.choice(text_pairs))

("Darling, we're going to forget all about these dreams and think about something cheerful, aren't we?", '[start] عزيزتي , سننسى كل هذه ال احلام و نفكر في شىء مرح , اليس كذلك ؟ [end]')
("But you can't murder me just like that!", '[start] لكن لو تقتلني ! فستكون تلك جريمة قتل [end]')


In [49]:
len(text_pairs)

35118

In [50]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[: num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]

In [51]:
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")

35118 total pairs
29851 training pairs
5267 validation pairs


#### Vectorizing the text data 

In [52]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


vocab_size = 20000
sequence_length = 50
batch_size = 265

def custom_standardization(input_string):
    return tf.strings.regex_replace(input_string, "[%s]" % re.escape(strip_chars), "")

eng_vectorization = TextVectorization(
            # max_tokens=vocab_size, 
            output_mode='int', 
            output_sequence_length=sequence_length)

ar_vectorization = TextVectorization(
    # max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
    max_tokens=vocab_size)

eng_texts = [pair[0] for pair in text_pairs]
ar_texts = [pair[1] for pair in text_pairs]
eng_vectorization.adapt(eng_texts)
ar_vectorization.adapt(ar_texts)

In [53]:
len(ar_vectorization.get_vocabulary()), len(eng_vectorization.get_vocabulary())

(20000, 13164)

In [54]:
len(ar_vectorization.get_vocabulary()), len(eng_vectorization.get_vocabulary())

(20000, 13164)

In [55]:
def format_dataset(eng, ar):
    eng = eng_vectorization(eng)
    ar = ar_vectorization(ar)
    return ({"encoder_inputs": eng, "decoder_inputs": ar[:, :-1],}, ar[:, 1:])


def make_dataset(pairs):
    eng_texts, ar_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ar_texts = list(ar_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ar_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [56]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [57]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (265, 50)
inputs["decoder_inputs"].shape: (265, 50)
targets.shape: (265, 50)


### Building the Model 

In [58]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):

      config = super().get_config().copy()
      config.update({
          'embed_dim': self.embed_dim,
          'dense_dim': self.dense_dim,
          'num_heads': self.num_heads,
      })
      return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, pretrained=False, weights=False, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        if not pretrained:
          self.token_embeddings = layers.Embedding(
              input_dim=vocab_size, output_dim=embed_dim
          )
        else:
          # pre-trained
          self.token_embeddings = layers.Embedding(
              input_dim=vocab_size, output_dim=embed_dim, weights=[weights]
          ) 

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
      
    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'sequence_length': self.sequence_length,
      'vocab_size': self.vocab_size,
      'embed_dim': self.embed_dim,
      })
      return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True


    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'embed_dim': self.embed_dim,
      'latent_dim': self.latent_dim,
      'num_heads': self.num_heads,
      })
      return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [61]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-06-08 02:17:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-08 02:17:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-08 02:17:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [65]:
import os
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), f"{os.getcwd()}/glove.6B.300d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [66]:
vocab = eng_vectorization.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

In [122]:
import gensim
import re
import numpy as np

In [193]:
def get_weights(vectorizer, embeddings_path, is_gensim=False):
    path_to_glove_file = embeddings_path
    
    if is_gensim:
        embeddings_index = gensim.models.Word2Vec.load(embeddings_path)
        
    else:
        embeddings_index = {}
        with open(path_to_glove_file) as f:
            for line in f:
                word, coefs = line.split(maxsplit=1)
                coefs = np.fromstring(coefs, "f", sep=" ")
                embeddings_index[word] = coefs
    
    if not is_gensim:
        print("Found %s word vectors." % len(dict(embeddings_index)))
    else:
        print("Found %s word vectors." % (embeddings_index.wv.vectors.shape[0]))

    
    
    vocab = vectorizer.get_vocabulary()
    word_index = dict(zip(vocab, range(len(vocab))))
    num_tokens = len(vocab)
    embedding_dim = 300
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        if not is_gensim:
            embedding_vector = embeddings_index.get(word)
        else:
            if word in embeddings_index.wv:
                embedding_vector = embeddings_index.wv[word]
            else:
                embedding_vector = None
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            embedding_matrix[i] = np.random.uniform(-.1, .1, size=(embedding_dim))
            misses += 1
    print("Converted %d words (%d misses)" % (hits, misses))
    return embedding_matrix

In [197]:
english_embeddings = get_weights(eng_vectorization, './glove.6B.300d.txt')

Found 400000 word vectors.
Converted 11896 words (1268 misses)


In [195]:
# ! wget https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_wiki.zip
# ! unzip -q full_grams_cbow_300_wiki.zip

In [196]:
arabic_embeddings = get_weights(ar_vectorization, './full_grams_cbow_300_wiki.mdl', is_gensim=True)

Found 662109 word vectors.
Converted 12646 words (7354 misses)


In [199]:
ar_vocab_size = len(ar_vectorization.get_vocabulary())
en_vocab_size = len(eng_vectorization.get_vocabulary())
en_vocab_size, ar_vocab_size

(13164, 20000)

In [201]:
embed_dim = 300
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, en_vocab_size, embed_dim, pretrained=True, weights=english_embeddings)(encoder_inputs)
# x = PositionalEmbedding(sequence_length, num_tokens, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, ar_vocab_size, embed_dim, pretrained=True, weights=arabic_embeddings)(decoder_inputs)
# x = PositionalEmbedding(sequence_length, ar_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(ar_vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [202]:
googledrive_path = './pretrained_both'

In [203]:
from keras import callbacks
early_stopping_cb = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(googledrive_path+'/weights_adam.ckpt', monitor='val_accuracy', save_weights_only=True,verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=googledrive_path+"/logs")
lr_schr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=True, factor=0.3, min_lr=0.0001)
cbs = [early_stopping_cb, checkpoint_cb, tensorboard_callback, lr_schr]

In [204]:
epochs = 100  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=cbs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding_8 (Positio (None, None, 300)    3964200     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder_4 (Transfor (None, None, 300)    4119848     positional_embedding_8[0][0]     
________________________________________________________________________________________

In [78]:
epochs = 100  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=cbs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding_4 (Positio (None, None, 300)    3964200     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder_2 (Transfor (None, None, 300)    4119848     positional_embedding_4[0][0]     
________________________________________________________________________________________

In [79]:
latest = tf.train.latest_checkpoint(googledrive_path)
transformer.load_weights(latest)

In [205]:
ar_vocab = ar_vectorization.get_vocabulary()
ar_index_lookup = dict(zip(range(len(ar_vocab)), ar_vocab))
max_decoded_sentence_length = sequence_length


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ar_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in val_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts[:30])
    translated = decode_sequence(input_sentence)
    print(input_sentence, '\n', translated)
    print('*'*50)

This was originally just a water container. 
 [start] هذه كانت ال حجرة سفينة فقط [UNK] [end]
**************************************************
Room 1 4. Next to the solarium. 
 [start] ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة [end]
**************************************************
Maybe not as thick as the ones that Joshua blew down with his trumpet. 
 [start] ربما لا بين ال ثلج كما [UNK] [end]
**************************************************
Room 1 4. Next to the solarium. 
 [start] ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة [end]
**************************************************
You're not Han Na, are you? 
 [start] انت لست هان نا ، اليس كذلك ؟ [end]
**************************************************
I Won't Be Here,And- 
 [start] لن اكون ساكون [end]
**************************************************
You're not Han Na, are you? 
 [start] انت لست هان نا ، اليس كذلك ؟ [end]
****************

In [82]:
def get_bleu():
  
  preds, src = [], []

  with tqdm(total=len(val_pairs), position=0, leave=True) as pbar:
    for en_sent, ar_sent in tqdm(val_pairs, position=0, leave=True):
      translated = decode_sequence(en_sent)
      preds.append(translated)
      src.append(ar_sent)
      pbar.update()

    return src, preds
    # print_scores(src, preds)



In [83]:
def print_scores(trgs, preds):
    print('----- Bleu-n Scores -----')
    print("1:", corpus_bleu(trgs, preds, weights=[1.0/1.0])*100)
    print("2:", corpus_bleu(trgs, preds, weights=[1.0/2.0, 1.0/2.0])*100)
    print("3:", corpus_bleu(trgs, preds, weights=[1.0/3.0, 1.0/3.0, 1.0/3.0])*100)
    print("4:", corpus_bleu(trgs, preds)*100)
    print('-'*25)

In [84]:
src, preds = get_bleu()

100%|██████████| 5267/5267 [18:20<00:00,  4.79it/s]


In [85]:
from nltk.translate.bleu_score import corpus_bleu

In [86]:
print_scores(preds, src)

----- Bleu-n Scores -----
1: 40.14637653331428


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


2: 63.3611683393814
3: 73.77039652394956
4: 79.59972885593355
-------------------------


In [87]:
for _ in range(10):
  i = random.randint(0, 500)
  print("prediction:", preds[i][7:-6])
  print("source:", src[i][7:-6])
  print('_'*100)

prediction:  اذا كان [UNK] على ال يدي
source:  لو وق بين يدى
____________________________________________________________________________________________________
prediction:  ماذا قلت ؟
source:  - ماذا قلت ؟
____________________________________________________________________________________________________
prediction:  يبدو انك [UNK]
source:  . واو .... رييس انت تكسب ال كثير من ال اموال
____________________________________________________________________________________________________
prediction:  لا ، لا ، لا
source:  لا يوجد رد
____________________________________________________________________________________________________
prediction:  هذه هي ال احذية لكن ، اعتقد انه في سريرك
source:  هذه امتعته ، لكن اعتقد هو في حجرته .
____________________________________________________________________________________________________
prediction:  لانى [UNK]
source:  #، هذا جزيي ال مفضل لانكم #
___________________________________________________________________________________________________

In [206]:
!zip translation_pre_both.zip ./pretrained_both/weights_adam.ckpt.data-00000-of-00001 ./pretrained_en/weights_adam.ckpt.index

  adding: pretrained_both/weights_adam.ckpt.data-00000-of-00001 (deflated 9%)
  adding: pretrained_en/weights_adam.ckpt.index (deflated 76%)


In [207]:
from IPython.display import FileLink
FileLink(r'translation_pre_both.zip')

/kaggle/working/translation_pre_both.zip